In [ ]:
#import libraries

In [ ]:
from datetime import timedelta
from stat import ST_DEV
from sys import argv
import numpy as np
import pandas as pd
import time
import math
import os
import kneed
import scipy as sp
import scipy.spatial
import scipy.sparse
import networkx as netx
import matplotlib.pyplot as plt
import sys

In [ ]:
#infinite global value

In [ ]:
greatest_value = -sys.maxsize - 1

In [ ]:
# clear the system

In [ ]:
clear = lambda: os.system('clear')

In [ ]:
#creates a list filled with zeros as long as the parameter

In [ ]:
def zerolist(n):
    list_zero = [0] * n
    return list_zero

In [ ]:
# finds the nodes,adjacency,edges and number of nodes attributes of graph

In [ ]:
def findCompGraph(graph):
    nodes = dict(graph.nodes())
    adjacency = dict(graph.adjacency())
    node_num = graph.number_of_nodes()
    edges = dict(graph.edges())
    return nodes, adjacency, edges, node_num

In [ ]:
#CutESC algorithm starts here and in the code below a spatial graph is generated using Delaunay graph

In [ ]:
def CutESC(data):
    tri = sp.spatial.Delaunay(data)                   # delaunay generator
    lil = sp.sparse.lil_matrix((tri.points, tri.points))     #turn the dealunay triangulars to matrix
    indices, indptr = tri.vertex_neighbor_vertices
    for k in range(tri.points):
        lil.rows[k] = indptr[indices[k]:indices[k + 1]]
        lil.data[k] = np.ones_like(lil.rows[k])               #example data of same shape as row that is filled with ones
    coo = lil.tocoo()                             #coo format to access to each node edge node pair
    conns = np.vstack((coo.row, coo.col)).T           #connections have two nodes at the ends of the edge
    delaunay_conns = np.sort(conns, axis=1)              # this is the edges(delaunay connnections)
    graph = netx.Graph(delaunay_conns)
    nodes = dict(graph.nodes())
    for i in nodes:
        nodes[i]['node'] = np.array(data[i])             #add nodes
    adjacency = dict(graph.adjacency())
    removed = []
    for i in nodes:                                      #add weights to nodes
        for j in adjacency[i]:
            weight = np.linalg.norm(nodes[i]['node'] - nodes[j]['node'])
            if weight != 0:
                graph.edges[(i, j)]['weight'] = weight
            else:
                removed.append((i, j))
    graph.remove_edges_from(removed)

In [ ]:
#In this part of the code gabriel graph is formed from delaunay triangulation

In [ ]:
    tree = scipy.spatial.cKDTree(data)
    c = tri.points[delaunay_conns]            # center of each edge
    m = (c[:, 0, :] + c[:, 1, :]) / 2            # midpoint of each edge
    r = np.sqrt(np.sum((c[:, 0, :] - c[:, 1, :]) ** 2, axis=1)) / 2            # radius
    n = tree.query(x=m, k=1)[0]                 #the closest point for each midpoint
    if n >= r * 0.999:
        g = n                         #closest point to midpoint is at a distance r, then it is a Gabriel edge
    gabriel_conns = delaunay_conns[g]  # gabriel edges

    graph2 = netx.Graph(gabriel_conns)
    nodes2 = dict(graph2.nodes())
    for i in nodes2:
        nodes2[i]['node'] = np.array(data[i])       #add nodes
    adjacency2 = dict(graph2.adjacency())
    removed2 = []
    for i in nodes2:                                  #add weights to nodes
        for j in adjacency2[i]:
            weight2 = np.linalg.norm(nodes2[i]['node'] - nodes2[j]['node'])
            if weight2 != 0:
                graph2.edges[(i, j)]['weight'] = weight2
            else:
                removed2.append((i, j))
    graph2.remove_edges_from(removed2)
    nodes2 = dict(graph2.nodes())
    adjacency2 = dict(graph2.adjacency())

In [ ]:
#In this part of the graph globally long edges are found and removed from the graph

In [ ]:
def gloEdges(graph):
    nodes, adjacency, edges, node_num = findCompGraph(graph)
    weights = list(netx.get_edge_attributes(graph, 'weight').values())
    mean_loc = zerolist(node_num)

    for i in nodes:  # the mean length of incident edges of vertex Pi
        loc_m = 0
        for j in adjacency[i]:
            loc_m = loc_m + graph.edges[i, j]["weight"]
        if len(adjacency[i]) == 0:
            mean_loc[i] = loc_m  # mean(Pi)
            continue
        else:
            loc_m = loc_m / len(adjacency[i])
            mean_loc[i] = loc_m  # mean(Pi)

    global_m = np.mean(weights)  # mean(GG)          #the mean length of all edges in gabriel graph
    glo_std = 0

    for i in nodes:  # the standard derivation of the mean length of edges in neigborhood
        glo_std = glo_std + math.pow(global_m - mean_loc[i], 2)
    glo_std = math.sqrt(glo_std / (node_num - 1))  # std(GG)
    GCuti = list()  # cut edge value

    for i in range(node_num):
        if mean_loc[i] == 0:
            GCuti.append(0)
        else:
            var = global_m * glo_std / mean_loc[i]
            var = var + global_m
            GCuti.append(var)

    remove = list()  # remove globally long edges

    for i in edges:
        if (edges[i]["weight"] >= GCuti[i[0]]) or (edges[i]["weight"]) >= GCuti[i[1]]:
            remove.append(i)
    graph.remove_edges_from(remove)

In [ ]:
#In this part locally long edges found and removed from the graph

In [ ]:
def findLoc(graph):
    nodes, adjacency, edges, node_num = findCompGraph(graph)
    mean_loc = list()
    loc_st = zerolist(node_num)
    label = list()
    graph_var = list()
    for var in sorted(netx.connected_components(graph), reverse=True):
        graph_var[var] = [var]
    for i in range(len(graph_var)):
        for j in graph_var[i]:
            label.insert(j, i)
    label = np.array(label)
    graph_var_m = zerolist(len(graph_var))
    nei_m = 0
    for i in nodes:  # the mean length of edges in the second order neighborhood of a vertex Pi in a subgraph Gx
        loc_m = 0
        nei_num = 0
        for j in adjacency[i]:
            weight = graph.edges[i, j]['weight']
            loc_m = loc_m + weight
            nei_num = nei_num + 1
            for n in adjacency[j]:
                weight = graph.edges[j, n]['weight']
                loc_m = loc_m + weight
                nei_num = nei_num + 1

        if nei_num == 0:
            nei_m = loc_m
            graph_var_m[label[i]] = nei_m + graph_var_m[label[i]]  # mean(Pi)
            continue
        if nei_num != 0:
            nei_m = loc_m / nei_num
            graph_var_m[label[i]] = nei_m + graph_var_m[label[i]]  # mean(Pi)

    for i in range(len(graph_var_m)):
        graph_var_m[i] = graph_var_m[i] / len(graph_var[i])  # mean(Gk)

    for i in nodes:  # the standard derivation of all edges that are directly connected to vertex Pi
        loc_std = 0
        nei_num = 0
        for j in adjacency[i]:
            weight = graph.edges[i, j]['weight']
            loc_std = loc_std + math.pow(graph_var_m[label[i]] - weight, 2)
            nei_num = nei_num + 1
            for n in adjacency[j]:
                weight = graph.edges[j, n]['weight']
                loc_std = loc_std + math.pow(graph_var_m[label[i]] - weight, 2)
                nei_num = nei_num + 1

        if nei_num == 0:
            loc_st[i] = loc_std  # std(Pi)
            continue
        if nei_num != 0:
            loc_std = math.sqrt(loc_std / (nei_num - 1))
        loc_st[i] = loc_std  # std(Pi)

    alpha = 1  # should be between 0 and 1 set to 1 by default
    remove = list()  # remove part
    for i in edges:
        val1 = math.exp(graph_var_m[label[i[0]]] / graph.edges[i]['weight'])
        val1 = alpha * loc_st[i[0]] * val1
        val1 = graph_var_m[label[i[0]]] + val1
        val2 = math.exp(graph_var_m[label[i[1]]] / graph.edges[i]['weight'])
        val2 = alpha * loc_st[i[1]] * val2
        val2 = graph_var_m[label[i[1]]] + val2
        if (edges[i]['weight'] >= val1) or (edges[i]['weight'] >= val2):
            remove.append(i)
    graph.remove_edges_from(remove)

In [ ]:
# In this section other locally long edges are identified and removed

In [ ]:
def findOtherLocal(graph):
    # finding other locally long edges
    nodes, adjacency, edges, node_num = findCompGraph(graph)
    loc_me = zerolist(node_num)
    loc_st = zerolist(node_num)
    for i in nodes:  # the mean length of edges in the second order neighborhood of a vertex Pi in a  new subgraph
        loc_m = 0
        nei_num = 0
        for j in adjacency[i]:
            weight = graph.edges[i, j]['weight']
            loc_m = loc_m + weight
            nei_num = nei_num + 1
            for n in adjacency[j]:
                weight = graph.edges[j, n]['weight']
                loc_m = loc_m + weight
                nei_num = nei_num + 1

        if nei_num == 0:
            nei_m = loc_m
            loc_me[i] = nei_m  # mean(Pi)
            continue
        if nei_num != 0:
            nei_m = loc_m / nei_num
            loc_me[i] = nei_m  # mean(Pi)

    for i in nodes:  # the standard derivation of all edges that are directly connected to vertex Pi
        loc_std = 0
        nei_num = 0
        for j in adjacency[i]:
            weight = graph.edges[i, j]['weight']
            loc_std = loc_std + math.pow(loc_me[i] - weight, 2)
            nei_num = nei_num + 1
            for n in adjacency[j]:
                weight = graph.edges[j, n]['weight']
                loc_std = loc_std + math.pow(loc_me[i] - weight, 2)
                nei_num = nei_num + 1

        if nei_num == 0:
            loc_st[i] = loc_std  # std(Pi)
            continue
        if nei_num != 0:
            if nei_num != 1:
                loc_std = math.sqrt(loc_std / (nei_num - 1))
            loc_st[i] = loc_std  # std(Pi)

    beta = 1  # should be between 0 and 1 set to 1 by default
    remove = list()  # remove part
    for i in edges:
        val1 = math.exp(loc_me[i[0]] / graph.edges[i]['weight'])
        val1 = beta * loc_st[i[0]] * val1
        val1 = loc_me[i[0]] + val1
        val2 = math.exp(loc_me[i[1]] / graph.edges[i]['weight'])
        val2 = beta * loc_st[i[1]] * val2
        val2 = loc_me[i[1]] + val2
        if (edges[i]['weight'] >= val1) or (edges[i]['weight'] >= val2):
            remove.append(i)
    graph.remove_edges_from(remove)


In [ ]:
#The part below assigns labels to clusters by using elbow method

In [ ]:
def assignClusterLabels(graph):
    num_node = graph.number_of_nodes()
    histog = list()
    label = zerolist(num_node)
    Scc = list()
    i = 0
    for var in sorted(netx.connected_components(graph), key=len, reverse=True):
        Scc[i] = var
        i = i + 1
    for i in range(len(Scc)):
        for j in Scc[i]:
            label[j] = i  # find strongly connected components in the graph and sort it by descending order

    for i in range(len(Scc)):  # finding number of vertices in each Scc connected components
        histog.append(len(Scc[i]))
    leng = len(histog)
    leng = list(range(leng))
    knees = kneed.KneeLocator(leng, histog, S=1.0, curve="convex", direction="decreasing")
    idx = knees.knee  # locate elbow points

    for i in range(idx, len(Scc)):
        for j in Scc[i]:
            label[j] = -1

    label = np.array(label)
    Scc = np.array(Scc)